In [1]:
!rm -rf /kaggle/working/multimodal-eq-sizing

In [2]:
!git clone https://github.com/brianrp09232000/multimodal-eq-sizing.git /kaggle/working/multimodal-eq-sizing

Cloning into '/kaggle/working/multimodal-eq-sizing'...
remote: Enumerating objects: 78, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 78 (delta 18), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (78/78), 15.03 KiB | 3.00 MiB/s, done.
Resolving deltas: 100% (18/18), done.


In [3]:
!pip install yfinance

# Download SPY as the base

In [4]:
import pandas as pd
import yfinance as yf

from datetime import datetime, timedelta

In [5]:
df_ticker_return = pd.read_csv("/kaggle/input/news-trading/return_data.csv")
df_ticker_return['Date'] = pd.to_datetime(df_ticker_return['Date'], utc=True)
df_ticker_return.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,ticker
0,2010-01-04 05:00:00+00:00,6.487649,6.520174,6.455732,6.505279,493729600,0.0,0.0,AAPL
1,2010-01-05 05:00:00+00:00,6.523214,6.553307,6.482178,6.516527,601904800,0.0,0.0,AAPL
2,2010-01-06 05:00:00+00:00,6.516527,6.542364,6.406185,6.412873,552160000,0.0,0.0,AAPL
3,2010-01-07 05:00:00+00:00,6.436583,6.444183,6.354511,6.401018,477131200,0.0,0.0,AAPL
4,2010-01-08 05:00:00+00:00,6.392506,6.444181,6.354814,6.443573,447610800,0.0,0.0,AAPL


In [6]:
grouped_ticker_by_date = df_ticker_return.groupby(["ticker"]).agg(['min', 'max', 'count'])["Date"]
grouped_ticker_by_date

,min,max,count
ticker,,,
AAPL,2010-01-04 05:00:00+00:00,2018-12-28 05:00:00+00:00,2263
ADBE,2010-01-04 05:00:00+00:00,2018-12-28 05:00:00+00:00,2263
AMZN,2010-01-04 05:00:00+00:00,2018-12-28 05:00:00+00:00,2263
FB,2012-05-18 04:00:00+00:00,2018-12-28 05:00:00+00:00,1664
GOOG,2010-01-04 05:00:00+00:00,2018-12-28 05:00:00+00:00,2263
JPM,2010-01-04 05:00:00+00:00,2018-12-28 05:00:00+00:00,2263
MSFT,2010-01-04 05:00:00+00:00,2018-12-28 05:00:00+00:00,2263
NFLX,2010-01-04 05:00:00+00:00,2018-12-28 05:00:00+00:00,2263
TSLA,2010-06-29 04:00:00+00:00,2018-12-28 05:00:00+00:00,2141


In [7]:
print(len(grouped_ticker_by_date))
print(grouped_ticker_by_date["min"].min(), grouped_ticker_by_date["min"].max())
print(grouped_ticker_by_date["max"].min(), grouped_ticker_by_date["max"].max())
print(grouped_ticker_by_date["count"].min(), grouped_ticker_by_date["count"].max())

10
2010-01-04 05:00:00+00:00 2012-05-18 04:00:00+00:00
2018-12-28 05:00:00+00:00 2018-12-28 05:00:00+00:00
1664 2263


In [8]:
start = grouped_ticker_by_date["min"].min()
end = grouped_ticker_by_date["max"].max()

In [9]:
ticker = "SPY"
ticker_yf = yf.Ticker(ticker)
df_base_ticker_return = ticker_yf.history(start = start, end = end+timedelta(days=1))
df_base_ticker_return['ticker']= ticker 
df_base_ticker_return.reset_index(inplace=True)

# Compute excess return

In [10]:
excess_return_formula = "(Close / Open) - 1"
df_ticker_return["o2c_return"] = df_ticker_return.eval(excess_return_formula)
df_base_ticker_return["spy_o2c_return"] = df_base_ticker_return.eval(excess_return_formula)
df_base_ticker_return = df_base_ticker_return[["Date", "spy_o2c_return"]]

In [11]:
df_with_target = pd.merge(df_ticker_return, df_base_ticker_return, on="Date", how="left")
df_with_target["excess_return"] = df_with_target.eval("o2c_return - spy_o2c_return")
df_with_target = df_with_target.drop(['o2c_return', 'spy_o2c_return'], axis=1)

In [12]:
df_with_target.to_csv('return_data_with_target.csv', index=False)